In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt  


import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  



import plotly.plotly as py
from plotly.graph_objs import *

# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout

 
# df=df_merged
# for column in df.columns:
#     for idx in df[column].index:
#         x = df.get_value(idx,column)
#         try:
#             x = unicode(x.encode('utf-8','ignore'),errors ='ignore') if type(x) == unicode else unicode(str(x),errors='ignore')
#             df.set_value(idx,column,x)
#         except Exception:
#             print 'encoding error: {0} {1}'.format(idx,column)
#             df.set_value(idx,column,'')
#             continue


########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



##### for getting geolocation data  and to calculate distance between two geolocations
import requests
import json
import geopy.distance   

In [18]:
dict_world_ranking_univ={}
list_files=["world_ranking_part1.csv","world_ranking_part2.csv" ]



with open("/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/University_rankings_and_names/world_ranking_part1.csv", "r") as ins:

    cont=0
    lista=[]

    for line in ins:
        cont +=1
       # print line

        if cont ==1:  # world ranking                       
            lista.append(line.replace('\n','').replace('\t',''))
        elif cont == 2: # university name
            univ=line.replace('\n','').replace('\t','')
            lista.append(univ)
        elif cont ==3: # national ranking
            lista.append(line.replace('\n','').replace('\t',''))

        elif cont == 4: pass
        elif cont ==5:

            dict_world_ranking_univ[univ]={"world_ranking":lista[0], "University":univ, "national_ranking":lista[2]}
            print univ, dict_world_ranking_univ[univ]
            cont =0
            lista=[]
 

with open("/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/University_rankings_and_names/world_ranking_part2.csv", "r") as ins:

    cont=0
    lista=[]

    for line in ins:
        cont +=1
       # print line

        if cont ==1:  # world ranking                       
            lista.append(line.replace('\n','').replace('\t',''))
        elif cont == 2: # university name
            univ=line.replace('\n','').replace('\t','').lower()
            lista.append(univ)
        elif cont ==3: # national ranking
            lista.append(line.replace('\n','').replace('\t',''))

        elif cont == 4: 
            dict_world_ranking_univ[univ]={"world_ranking":lista[0], "University":univ, "national_ranking":lista[2]}
            print univ, dict_world_ranking_univ[univ]
            cont =0
            lista=[]           
       
df= pd.DataFrame.from_dict(dict_world_ranking_univ, orient='index')

  
df.head()    
                       
# filename_output='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/University_rankings_and_names/world_ranking_parsed.csv'
# output=open(filename_output,'wt')
# print >> output, "World_Ranking", "University", "National_ranking"
# for item in sorted(dict_world_ranking_univ.keys()):
#     print >> output, item, dict_world_ranking_univ[item]["University"], dict_world_ranking_univ[item]["national_ranking"]

# output.close()
# print "written:", filename_output

df.to_csv("/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/University_rankings_and_names/world_ranking_parsed.csv", sep=',')
print "written:","/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/University_rankings_and_names/world_ranking_parsed.csv"


Harvard University {'world_ranking': '1', 'University': 'Harvard University', 'national_ranking': '1'}
Stanford University {'world_ranking': '2', 'University': 'Stanford University', 'national_ranking': '2'}
University of California, Berkeley {'world_ranking': '3', 'University': 'University of California, Berkeley', 'national_ranking': '3'}
University of Cambridge {'world_ranking': '4', 'University': 'University of Cambridge', 'national_ranking': '1'}
Massachusetts Institute of Technology (MIT) {'world_ranking': '5', 'University': 'Massachusetts Institute of Technology (MIT)', 'national_ranking': '4'}
Princeton University {'world_ranking': '6', 'University': 'Princeton University', 'national_ranking': '5'}
University of Oxford {'world_ranking': '7', 'University': 'University of Oxford', 'national_ranking': '2'}
California Institute of Technology {'world_ranking': '8', 'University': 'California Institute of Technology', 'national_ranking': '6'}
Columbia University {'world_ranking': '9',

In [20]:
df.head(100)

,world_ranking,University,national_ranking
Aarhus University,65,Aarhus University,2
Boston University,75,Boston University,39
Brown University,90,Brown University,45
California Institute of Technology,8,California Institute of Technology,6
Carnegie Mellon University,68,Carnegie Mellon University,36
Columbia University,9,Columbia University,7
Cornell University,13,Cornell University,11
Duke University,25,Duke University,19
Ecole Normale Superieure - Paris,87,Ecole Normale Superieure - Paris,3
Georgia Institute of Technology,93,Georgia Institute of Technology,47
